<a href="https://colab.research.google.com/github/fabiomatricardi/AI-RewriteText/blob/main/notebooks/ITALIAN_Vicuna13b_1_5_GGUF_GPTQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ITALIAN Vicuna13b TESTS

---

After seeing Vicuna33b replying to Italian questions on https://huggingface.co/spaces/lmsys/chatbot-arena

## References
Chatbot Arena
Evaluating large language model (LLM) based chat assistants is challenging due to their broad capabilities and the inadequacy of existing benchmarks in measuring human preferences. To address this, we explore using strong LLMs as judges to evaluate these models on more open-ended questions.


### Prompt Battle judge
- uses an LLM to judge the prompts https://arxiv.org/pdf/2306.05685.pdf
Judging LLM-as-a-judge with MT-Bench and Chatbot Arena
https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard


In [ ]:
!wget https://github.com/fabiomatricardi/AI-RewriteText/raw/main/texts/tomp3.cc%20-%20Mistral%207B%20%20The%20New%207B%20LLaMA%20Killer_1080p.txt
!wget https://github.com/fabiomatricardi/AI-RewriteText/raw/main/texts/tomp3.cc%20-%20This%20AI%20Startup%20ACTUALLY%20Opens%20AI%20%20starts%20with%20POWERFUL%207B%20LLM_1080p.txt

# GGML section
using TheBloke/vicuna-13B-v1.5-GGML


In [ ]:
%%capture
!pip install ctransformers>=0.2.24
!pip install langchain
!pip install rich
!pip install transformers

In [ ]:
!wget https://huggingface.co/TheBloke/vicuna-13B-v1.5-GGML/resolve/main/vicuna-13b-v1.5.ggmlv3.q4_K_M.bin

--2023-10-13 23:38:23--  https://huggingface.co/TheBloke/vicuna-13B-v1.5-GGML/resolve/main/vicuna-13b-v1.5.ggmlv3.q4_K_M.bin
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.124, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/7e/81/7e818b49dedd66a09da83a8729191a51055937ab6219c73cbc2c966a7427d215/5fe5ec7c23d08a961cd0e1bbc4944ba36254c887a50a5d94ce173c488da10a9e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27vicuna-13b-v1.5.ggmlv3.q4_K_M.bin%3B+filename%3D%22vicuna-13b-v1.5.ggmlv3.q4_K_M.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1697499503&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NzQ5OTUwM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83ZS84MS83ZTgxOGI0OWRlZGQ2NmEwOWRhODNhODcyOTE5MWE1MTA1NTkzN2FiNjIxOWM3M2NiY

In [ ]:
#@title Restart Runtime {display-mode: "form"}
import ipywidgets as widgets
def restart(b):
  exit()

button2 = widgets.Button(
    description='Restart Runtime',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
button2.on_click(restart)
button2


Button(button_style='warning', description='Restart Runtime', icon='check', style=ButtonStyle(), tooltip='Clic…

In [ ]:
from rich import print
from tqdm.rich import trange, tqdm
from rich import console
from rich.panel import Panel
from rich.markdown import Markdown
from rich.text import Text
import warnings
warnings.filterwarnings(action='ignore')
from ctransformers import AutoModelForCausalLM, AutoConfig, Config
import datetime
from rich.console import Console
console = Console(width=110)

In [ ]:
conf = AutoConfig(Config(temperature=0.7, repetition_penalty=1.1, batch_size=52,
                max_new_tokens=1024, context_length=1024))
llm = AutoModelForCausalLM.from_pretrained("/content/vicuna-13b-v1.5.ggmlv3.q4_K_M.bin",
                                           model_type="llama", config = conf)

In [ ]:
# FUNCTION TO LOG ALL CHAT MESSAGES INTO chathistory.txt
def writehistory(text):
    with open('20231014_italian_Vicuna13b-history.txt', 'a') as f:
        f.write(text)
        f.write('\n')
    f.close()

def vicunaQ4KM_CT(prompt):
  from rich.markdown import Markdown
  #t_vicuna = f"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {prompt} ASSISTANT:"
  t_vicuna = f"USER: {prompt} ASSISTANT:"
  start = datetime.datetime.now()
  console.print(f"[italic bold bright_red]Prompt: {prompt}")
  answer = llm(t_vicuna, temperature = 0.7, repetition_penalty = 1.15,
             max_new_tokens = 2048)
  stop = datetime.datetime.now()
  tok2 = len(llm.tokenize(t_vicuna))
  tok1 = len(llm.tokenize(answer))
  #console.print(f"[italic]Number of characters in orginal prompt: {len(prompt)}") #it works with CTransformers without Langchain
  console.print(f"[italic bold]Number of tokens in the  prompt: {tok2}") #it works with CTransformers without Langchain
  console.print(f"[italic bold]Number of tokens in the  answer: {tok1}")
  console.print(Markdown(answer))
  console.print(f"[bold italic green] Generated by Vicuna-13b in {stop-start}")
  text = f"user: {prompt}\nVicuna-7b: {answer}\nGenerated in {stop-start}"
  writehistory(text)
  console.print(f"[blue1] ---")
  return answer

### Test with Itlian prompts

In [ ]:
from rich.prompt import Prompt

In [ ]:
database = []

In [ ]:
question = input("La tua domanda: ")
prompt1 = vicunaQ4KM_CT(question)
res = {'question': question,
       'answer': prompt1}
database.append(res)

La tua domanda: cosa significa scienza?


Prompt: cosa significa scienza?

Number of tokens in the  prompt: 13

Number of tokens in the  answer: 170

La scienza è un insieme di metodologie e conoscenze che permettono di comprendere il funzionamento del mondo  
naturale, attraverso l'osservazione, la sperimentazione e l'analisi dei dati. La scienza si basa              
sull'empirismo, ovvero sulla raccolta di dati osservativi, che vengono poi analizzati e interpretati per      
formulate teorie e leggi scientifiche. La scienza è utilizzata in molte aree della vita quotidiana, dalla     
medicina alla tecnologia, dall'ingegneria all'ambiente, e rappresenta uno degli strumenti più importanti per  
comprendere il mondo che ci circonda e per risolvere i problemi attuali.

 Generated by Vicuna-7b in 0:03:00.437367

 ---

In [ ]:
question = """Quali sono le relazione tra AI e la filosofia per il futuro?
Rispondi seguendo questo schema:
RIFLESSIONI: // I tuoi pensieri qui
CRITICHE: // Le critiche alle tue considerazioni
RISPOSTA: // La tua risposta finale"""
prompt1 = vicunaQ4KM_CT(question)
res = {'question': question,
       'answer': prompt1}
database.append(res)

Prompt: Quali sono le relazione tra AI e la filosofia per il futuro?
Rispondi seguendo questo schema:
RIFLESSIONI: // I tuoi pensieri qui
CRITICHE: // Le critiche alle tue considerazioni
RISPOSTA: // La tua risposta finale

Number of tokens in the  prompt: 78

Number of tokens in the  answer: 369

Riflessioni: L'AI ha una grande capacità di elaborazione delle informazioni e di apprendimento automatico, che
la rende in grado di eseguire compiti complessi e di soluzione di problemi. Tuttavia, l'AI è soggetta a       
limitazioni e errori, come la possibilità di avere pregiudizi o di commettere errori di valutazione. Inoltre, 
l'uso dell'AI può avere implicazioni etiche e sociali che devono essere prese in considerazione. Critiche: La 
relazione tra AI e filosofia per il futuro è complessa e multifacetata. Mentre alcuni filoni della filosofia, 
come l'etica e la teoria dell'intelligenza artificiale, sono direttamente coinvolti nell'analisi delle        
implicazioni etiche e sociali dell'AI, altri sembrano meno interessati a questa tematica. Inoltre, alcune     
critiche sostengono che la filosofia non ha alcun ruolo da svolgere nell'elaborazione dell'AI, in quanto le   
sue capacità sono limitate dalla sua natura tecnologica e scientifica. Risposta: La relazione tra AI e        
filosofia per il futuro è cruciale per affrontare le sfide etiche e sociali che l'uso dell'AI sta creando. La 
filosofia può contribuire all'analisi delle implicazioni della tecnologia sull'umanità, fornendo un quadro    
teorico per comprendere i problemi etici e sociali associati all'AI. Inoltre, la filosofia può aiutare a      
guidare il desenvol

 Generated by Vicuna-7b in 0:06:56.437224

 ---

In [ ]:
question = """Quali sono le relazione tra AI e apprendimento per il futuro?
Rispondi seguendo questo schema:
RIFLESSIONI: // I tuoi pensieri qui

CRITICHE: // Le critiche alle tue considerazioni

RISPOSTA: // La tua risposta finale
"""
prompt1 = vicunaQ4KM_CT(question)
res = {'question': question,
       'answer': prompt1}
database.append(res)

Prompt: Quali sono le relazione tra AI e apprendimento per il futuro?
Rispondi seguendo questo schema:
RIFLESSIONI: // I tuoi pensieri qui

CRITICHE: // Le critiche alle tue considerazioni

RISPOSTA: // La tua risposta finale

Number of tokens in the  prompt: 81

Number of tokens in the  answer: 429

RIFLESSIONI: L'AI e l'apprendimento automatico sono tecnologie in continua evoluzione che hanno il potenziale 
di trasformare molte aree della vita umana, dalle decisioni finanziarie alle cure mediche. Tuttavia, come     
tutte le tecnologie, anche l'AI ha i suoi limiti e rischi. È importante considerare gli effetti a lungo       
termine dell'uso dell'AI, comprese le questioni etiche che si presentano in questa area.                      

CRITICHE: Una critica all'idea che l'AI possa sostituire completamente i dati di mercato è che l'AI non può   
ancora prevedere il futuro con precisione e che ci sono molte variabili imprevisibili che possono influenzare 
le decisioni finanziarie. Inoltre, anche se l'AI può fornire valutazioni e raccomandazioni, la decisione      
finale spetta sempre all'uomo.                                                                                

RISPOSTA: Sono d'accordo con entrambe le posizioni presentate, ma credo che l'AI possa avere un impatto       
significativo sulla formazione del futuro. L'AI può essere utilizzata per analizzare grandi quantità di dati e
prevedere tendenze e cambiamenti nel mercato finanziario a breve e lungo termine, ma non può sostituire       
completamente la comprensione umana delle decisioni finanziarie. Inoltre, l'AI può essere utilizzata per      
creare modelli di simulazione che consentono agli investitori di esercitarsi e migliorare le loro capacità di 
previsione, senza dover affrontare i rischi reali del mercato finanziario. Tuttavia, è importante considerare 
anche gli effetti a lungo termine dell'uso dell'AI in questo campo e affrontare le questioni etiche che si    
presentano in questa area.

 Generated by Vicuna-7b in 0:07:59.796733

 ---

In [ ]:
question = """Traduci in lingua Inglese il seguente testo delimitato da tre backquotes:
```
RIFLESSIONI: L'AI e l'apprendimento automatico sono tecnologie in continua evoluzione che hanno il potenziale
di trasformare molte aree della vita umana, dalle decisioni finanziarie alle cure mediche. Tuttavia, come
tutte le tecnologie, anche l'AI ha i suoi limiti e rischi. È importante considerare gli effetti a lungo
termine dell'uso dell'AI, comprese le questioni etiche che si presentano in questa area.

CRITICHE: Una critica all'idea che l'AI possa sostituire completamente i dati di mercato è che l'AI non può
ancora prevedere il futuro con precisione e che ci sono molte variabili imprevisibili che possono influenzare
le decisioni finanziarie. Inoltre, anche se l'AI può fornire valutazioni e raccomandazioni, la decisione
finale spetta sempre all'uomo.

RISPOSTA: Sono d'accordo con entrambe le posizioni presentate, ma credo che l'AI possa avere un impatto
significativo sulla formazione del futuro. L'AI può essere utilizzata per analizzare grandi quantità di dati e
prevedere tendenze e cambiamenti nel mercato finanziario a breve e lungo termine, ma non può sostituire
completamente la comprensione umana delle decisioni finanziarie. Inoltre, l'AI può essere utilizzata per
creare modelli di simulazione che consentono agli investitori di esercitarsi e migliorare le loro capacità di
previsione, senza dover affrontare i rischi reali del mercato finanziario. Tuttavia, è importante considerare
anche gli effetti a lungo termine dell'uso dell'AI in questo campo e affrontare le questioni etiche che si
presentano in questa area.
```
"""
prompt1 = vicunaQ4KM_CT(question)
res = {'question': question,
       'answer': prompt1}
database.append(res)

Prompt: Traduci in lingua Inglese il seguente testo delimitato da tre backquotes:
```
RIFLESSIONI: L'AI e l'apprendimento automatico sono tecnologie in continua evoluzione che hanno il potenziale 
di trasformare molte aree della vita umana, dalle decisioni finanziarie alle cure mediche. Tuttavia, come     
tutte le tecnologie, anche l'AI ha i suoi limiti e rischi. È importante considerare gli effetti a lungo       
termine dell'uso dell'AI, comprese le questioni etiche che si presentano in questa area.                      

CRITICHE: Una critica all'idea che l'AI possa sostituire completamente i dati di mercato è che l'AI non può   
ancora prevedere il futuro con precisione e che ci sono molte variabili imprevisibili che possono influenzare 
le decisioni finanziarie. Inoltre, anche se l'AI può fornire valutazioni e raccomandazioni, la decisione      
finale spetta sempre all'uomo.                                                                                

RISPOSTA: Sono d'accordo con entrambe le posizioni presentate, ma credo che l'AI possa avere un impatto       
significativo sulla formazione del futuro. L'AI può essere utilizzata per analizzare grandi quantità di dati e
prevedere tendenze e cambiamenti nel mercato finanziario a breve e lungo termine, ma non può sostituire       
completamente la comprensione umana delle decisioni finanziarie. Inoltre, l'AI può essere utilizzata per      
creare modelli di simulazione che consentono agli investitori di esercitarsi e migliorare le loro capacità di 
previsione, senza dover affrontare i rischi reali del mercato finanziario. Tuttavia, è importante considerare 
anche gli effetti a lungo termine dell'uso dell'AI in questo campo e affrontare le questioni etiche che si    
presentano in questa area. 
```

Number of tokens in the  prompt: 505

Number of tokens in the  answer: 290

REFLECTIONS: Artificial intelligence (AI) and machine learning are constantly evolving technologies that have 
the potential to transform many aspects of human life, from financial decisions to medical care. However, like
all technologies, AI also has its limits and risks. It is important to consider the long-term effects of using
AI, including ethical issues that arise in this area.                                                         

CRITICISM: One criticism of the idea that AI can completely replace market data is that AI cannot yet predict 
the future with precision, and there are many unpredictable variables that can influence financial decisions. 
Furthermore, even if AI can provide evaluations and recommendations, the final decision ultimately lies with  
humans.                                                                                                       

RESPONSE: I agree with both positions presented, but I believe that AI can have a significant impact on       
shaping the future. AI can be used to analyze large amounts of data and predict trends and changes in the     
financial market at short and long term, but it cannot completely replace human understanding of financial    
decisions. Additionally, AI can be used to create simulation models that allow investors to practice and      
improve their prediction skills without having to face real risks in the financial market. However, it is     
important to consider even the long-term effects of using AI in this field and to address ethical issues that 
arise in this area.

 Generated by Vicuna-7b in 0:11:15.467698

 ---

In [ ]:
question = """Translate into Italian the following text delimited by triple backquotes:
```Lost in the Middle: How Language Models Use Long Contexts. Language models have become an important and flexible building block in a variety of user-facing language technologies, including conversational interfaces, search and summarization, and collaborative writing. These models perform downstream tasks primarily via prompting: all relevant task specification and data to process is formatted as a textual context, and the model returns a generated text completion. These input contexts can contain thousands of tokens, especially when using language models on lengthy inputs (e.g., legal or scientific documents, conversation histories, etc.) or augmenting them with external information (e.g.,relevant documents from a search engine, database query results, etc; Petroni et al., 2020; Ram et al., 2023; Shi et al., 2023; Mallen et al., 2023; Schick et al., 2023, inter alia). Handling these ...arly visualized in Figure 1, as we vary the position of the relevant information —language model performance is highest when relevant information occurs at the very beginning or end of its input context, and performance significantly degrades when models must access and use information in the middle of their input context (§3.3). For example, when relevant information is placed in the middle of its input context, GPT3.5-Turbo’s performance on the multi-document question task is lower than its performance when predicting without any documents (i.e., the closedbook setting; 56.1%).
```
"""
prompt1 = vicunaQ4KM_CT(question)
res = {'question': question,
       'answer': prompt1}
database.append(res)

Prompt: Translate into Italian the following text delimited by triple backquotes:
```Lost in the Middle: How Language Models Use Long Contexts. Language models have become an important and 
flexible building block in a variety of user-facing language technologies, including conversational 
interfaces, search and summarization, and collaborative writing. These models perform downstream tasks 
primarily via prompting: all relevant task specification and data to process is formatted as a textual 
context, and the model returns a generated text completion. These input contexts can contain thousands of 
tokens, especially when using language models on lengthy inputs (e.g., legal or scientific documents, 
conversation histories, etc.) or augmenting them with external information (e.g.,relevant documents from a 
search engine, database query results, etc; Petroni et al., 2020; Ram et al., 2023; Shi et al., 2023; Mallen 
et al., 2023; Schick et al., 2023, inter alia). Handling these ...arly visualized in Figure 1, as we vary the 
position of the relevant information —language model performance is highest when relevant information occurs 
at the very beginning or end of its input context, and performance significantly degrades when models must 
access and use information in the middle of their input context (§3.3). For example, when relevant information
is placed in the middle of its input context, GPT3.5-Turbo’s performance on the multi-document question task 
is lower than its performance when predicting without any documents (i.e., the closedbook setting; 56.1%).
```

Number of tokens in the  prompt: 368

Number of tokens in the  answer: 431

                                                                                                              
 Perduti nel bel mezzo: come i modelli di linguaggio utilizzano contesti lunghi. I modelli di linguaggio sono 
 diventati un importante e flessibile blocco costruttivo in una varietà di tecnologie del linguaggio utente,  
 compresi gli interfacce conversative, la ricerca e la riassunzione, e la scrittura collaborativa. Questi     
 modelli eseguono attività downstream principalmente tramite l'invio di messaggi: tutti i requisiti di task e 
 dati da elaborare sono formattati come contesto testuale, e il modello restituisce un completamento di testo 
 generato. Questi contesti di input possono contenere migliaia di token, specialmente quando si utilizzano    
 modelli di linguaggio su input di lunghezza (ad esempio, documenti legali o scientifici, historie delle      
 conversazioni, ecc.) o li arricchiscono con informazioni esterne (ad esempio, documenti rilevanti da un moto 
 di ricerca, risultati di query di database, ecc.). Il trattamento di questi ... visualizzato in Figure 1, co 
 variamo la posizione dell'informazione relevante — il rendimento del modello di linguaggio è massimo quando  
 l'informazione relevante si trova all'inizio o alla fine del suo contesto di input, e il rendimento si degra 
 significativamente quando i modelli devono accedere e utilizzare le informazioni nel bel mezzo del loro      
 contesto di input (§3.3). Ad esempio, quando l'informazione relevante viene posizionata nel bel mezzo del su 
 contesto di input, il rendimento di GPT3.5-Turbo sulla task della domanda multi-documento è inferiore al suo 
 rendimento quando prevede senza alcun documento (cioè il setting chiuso; 56,1%).                             
                                                                                                              

 Generated by Vicuna-7b in 0:12:08.699118

 ---

In [ ]:
question = """Fai una predizione sulle conseguenze derivanti dall'utilizzo della Intelligenza Artificiale nel campo medico.
Rispondi creando una lista in markdown evidenziando i punti chiave.

LISTA:
"""
prompt1 = vicunaQ4KM_CT(question)
res = {'question': question,
       'answer': prompt1}
database.append(res)

Prompt: Fai una predizione sulle conseguenze derivanti dall'utilizzo della Intelligenza Artificiale nel campo 
medico.
Rispondi creando una lista in markdown evidenziando i punti chiave.

LISTA:

Number of tokens in the  prompt: 65

Number of tokens in the  answer: 315

 1 Miglioramento delle prestazioni cliniche: l'IA può aiutare a migliorare la precisione e la velocità nel    
   diagnostica, monitoraggio e trattamento delle malattie.                                                    
 2 Personalizzazione dei trattamenti: grazie all'analisi dei dati individuali, l'IA può consentire una terapia
   più mirata ed efficace per ogni paziente.                                                                  
 3 Costi sostenibili: l'uso dell'IA in medicina potrebbe portare a riduzioni dei costi sanitari grazie alla   
   gestione del personale e delle risorse più efficiente.                                                     
 4 Prevenzione delle malattie: l'IA può aiutare a prevedere ed evitare le malattie croniche attraverso il     
   monitoraggio e l'analisi dei dati di salute in tempo reale.                                                
 5 Incremento della conoscenza scientifica: l'IA può aiutare a identificare nuove connessioni tra i dati      
   medici, portando allo sviluppo di nuovi farmaci ed interventi terapeutici.                                 
 6 Protezione dei dati personali: l'IA può essere utilizzata per migliorare la sicurezza e la privacy dei dati
   sanitari attraverso l'implementazione di algoritmi di crittografia e di altre tecnologie di sicurezza.

 Generated by Vicuna-7b in 0:05:59.156463

 ---

### save the logs into a PICKLE file

ou can use the pickle module for that. This module has two methods,

Pickling(dump): Convert Python objects into a string representation.
Unpickling(load): Retrieving original objects from a stored string representation.
https://docs.python.org/3.3/library/pickle.html

Code:
```
>>> import pickle
>>> l = [1,2,3,4]
>>> with open("test", "wb") as fp:   #Pickling
...   pickle.dump(l, fp)
...
>>> with open("test", "rb") as fp:   # Unpickling
...   b = pickle.load(fp)
...
>>> b
[1, 2, 3, 4]
```

Also Json

dump/dumps: Serialize
load/loads: Deserialize
https://docs.python.org/3/library/json.html

Code:
```
>>> import json
>>> with open("test", "w") as fp:
...     json.dump(l, fp)
...
>>> with open("test", "r") as fp:
...     b = json.load(fp)
...
>>> b
[1, 2, 3, 4]
```

In [ ]:
###############################
# save the database
###############################
import pickle
with open("databseVicuna13bITA", "wb") as fp:   #Pickling
  pickle.dump(database, fp)


In [ ]:
###############################
# load the database from pickle file
###############################
import pickle
with open("databseVicuna13bITA", "rb") as fp:   # Unpickling
  databaseVic13bITA = pickle.load(fp)

In [ ]:
databaseVic13bITA